In [150]:
import cv2 
import numpy as np 
import os
import pandas as pd


In [151]:
folder_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Mushmom_Classification"

In [152]:
files = os.listdir(folder_path)

In [153]:
images = [file for file in files if file.endswith('.JPG') or file.endswith('.png')]

In [154]:
def image_to_array(folder_path):
    files = os.listdir(folder_path)
    images = [file for file in files if file.endswith('.JPG') or file.endswith('.png')]
    image_list = [] 
    for image in images:
        img = cv2.imread(os.path.join(folder_path, image))
        image_list.append(img)
    return image_list

In [155]:
#Mushmom image
mushmom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Mushmom_Classification"
mushmom_images = image_to_array(mushmom_path)

In [156]:
#Horny_Mushroom 
horny_mushroom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Horny_Mushroom_Classification"
horny_mushroom_images = image_to_array(horny_mushroom_path)

In [157]:
#Poisonous_mushroom
poisonous_mushroom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Poisonous_Mushroom_Classification"
poisonous_mushroom_images = image_to_array(poisonous_mushroom_path)

In [158]:
#Zombie_mushmom
zombie_mushmom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Zombie_Mushmom_Classification"
zombie_mushmom_images = image_to_array(zombie_mushmom_path)

In [159]:
#Blue_Mushmom
blue_mushmom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Blue_Mushmom_Classification"
blue_mushmom_images = image_to_array(blue_mushmom_path)

In [160]:
#Green Mushroom
green_mushmom_path = "C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier/Green_Mushroom_Classification"
green_mushmom_images = image_to_array(green_mushmom_path)

In [161]:
mushmom_df = pd.DataFrame({'array_val' : mushmom_images , "actual_labels" : ['mushmom'] * len(mushmom_images) })

In [162]:
horny_mushroom_df = pd.DataFrame({'array_val' : horny_mushroom_images , "actual_labels" : ['horny_mushroom'] * len(horny_mushroom_images) })

In [163]:
poisonous_mushroom_df = pd.DataFrame({'array_val' : poisonous_mushroom_images , "actual_labels" : ['poisonous_mushroom'] * len(poisonous_mushroom_images) })

In [164]:
zombie_mushmom_df = pd.DataFrame({'array_val' : zombie_mushmom_images, 'actual_labels' : ['zombie_mushmom'] * len(zombie_mushmom_images)})

In [165]:
blue_mushmom_df = pd.DataFrame({'array_val' : blue_mushmom_images, 'actual_labels' : ['blue_mushmom'] * len(blue_mushmom_images)})

In [166]:
green_mushmom_df = pd.DataFrame({'array_val' : green_mushmom_images, 'actual_labels' : ['green_mushmom'] * len(green_mushmom_images)})

In [168]:
pd.concat([mushmom_df,horny_mushroom_df,poisonous_mushroom_df,zombie_mushmom_df,blue_mushmom_df,green_mushmom_df],axis=0).reset_index(drop=True)

SyntaxError: invalid syntax (430016101.py, line 1)

In [95]:
horny_mushroom_df

,array_val,actual_labels
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",horny_mushroom
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",horny_mushroom
2,"[[[248, 248, 248], [248, 248, 248], [248, 248,...",horny_mushroom
3,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",horny_mushroom
4,"[[[45, 95, 107], [48, 98, 110], [50, 100, 112]...",horny_mushroom
5,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",horny_mushroom
6,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",horny_mushroom
7,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",horny_mushroom
8,"[[[213, 174, 122], [213, 174, 122], [213, 174,...",horny_mushroom
9,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",horny_mushroom


In [67]:
img = data['array_val']

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state = 42)

In [69]:
import matplotlib.pyplot as plt 
plt.imshow(cv2.cvtColor(img[0], cv2.COLOR_BGR2RGB))
plt.show()